<a href="https://colab.research.google.com/github/LC1332/prompt-ng-andrew/blob/main/content/7.%20%E6%96%87%E6%9C%AC%E6%89%A9%E5%B1%95%20Expanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第七章 扩展

扩展是将短文本，例如一组说明或主题列表，输入到大型语言模型中，让模型生成更长的文本，例如基于某个主题的电子邮件或论文。这样做有一些很好的用途，例如将大型语言模型用作头脑风暴的伙伴。但这种做法也存在一些问题，例如某人可能会使用它来生成大量垃圾邮件。因此，当你使用大型语言模型的这些功能时，请仅以负责任的方式和有益于人们的方式使用它们。

在本章中，你将学会如何基于 OpenAI API 生成适用于每个客户评价的客户服务电子邮件。我们还将使用模型的另一个输入参数称为温度，这种参数允许您在模型响应中变化探索的程度和多样性。



# 吴恩达老师的Prompt Engineering课程学习

本项目由李鲁鲁老师在DataWahle翻译的中文版本的吴恩达老师与 OpenAI 合作推出的 《ChatGPT Prompt Engineering for Developers》基础上，进行了一些自己的实践思考。

原来项目的链接是 [https://github.com/datawhalechina/prompt-engineering-for-developers](https://github.com/datawhalechina/prompt-engineering-for-developers)

每一课都在colab上进行了改造，并将链接放在了readme，可以直接用colab进行学习。

In [ ]:
! pip install openai
! pip install python-dotenv

## 一、环境配置

同以上几章，你需要类似的代码来配置一个可以使用 OpenAI API 的环境

In [ ]:
# 将自己的 API-KEY 导入系统环境变量
# !export OPENAI_API_KEY='api-key'

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
# 导入第三方库

# _ = load_dotenv(find_dotenv())
# 读取系统中的环境变量

openai.api_key  = 'sk-' # 在这里输入你的OpenAI API Token
# 设置 API_KEY

In [3]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 温度系数
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


## 二、定制客户邮件

我们将根据客户评价和情感撰写自定义电子邮件响应。因此，我们将给定客户评价和情感，并生成自定义响应即使用 LLM 根据客户评价和评论情感生成定制电子邮件。

我们首先给出一个示例，包括一个评论及对应的情感

In [4]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [5]:
# 我们可以在推理那章学习到如何对一个评论判断其情感倾向
sentiment = "negative"

# 一个产品的评价
review = f"""
他们在11月份的季节性销售期间以约49美元的价格出售17件套装，折扣约为一半。\
但由于某些原因（可能是价格欺诈），到了12月第二周，同样的套装价格全都涨到了70美元到89美元不等。\
11件套装的价格也上涨了大约10美元左右。\
虽然外观看起来还可以，但基座上锁定刀片的部分看起来不如几年前的早期版本那么好。\
不过我打算非常温柔地使用它，例如，\
我会先在搅拌机中将像豆子、冰、米饭等硬物研磨，然后再制成所需的份量，\
切换到打蛋器制作更细的面粉，或者在制作冰沙时先使用交叉切割刀片，然后使用平面刀片制作更细/不粘的效果。\
制作冰沙时，特别提示：\
将水果和蔬菜切碎并冷冻（如果使用菠菜，则轻轻煮软菠菜，然后冷冻直到使用；\
如果制作果酱，则使用小到中号的食品处理器），这样可以避免在制作冰沙时添加太多冰块。\
大约一年后，电机发出奇怪的噪音，我打电话给客服，但保修已经过期了，所以我不得不再买一个。\
总的来说，这些产品的总体质量已经下降，因此它们依靠品牌认可和消费者忠诚度来维持销售。\
货物在两天内到达。
"""

我们已经使用推断课程中学到的提取了情感，这是一个关于搅拌机的客户评价，现在我们将根据情感定制回复。

这里的指令是：假设你是一个客户服务AI助手，你的任务是为客户发送电子邮件回复，根据通过三个反引号分隔的客户电子邮件，生成一封回复以感谢客户的评价。

In [6]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear valued customer,

Thank you for taking the time to leave a review about your recent purchase. We are sorry to hear that you were not completely satisfied with your experience.

We apologize for any inconvenience caused by the price increase of the product you purchased. We strive to provide our customers with the best possible prices, and we understand how frustrating it can be when prices change unexpectedly.

We appreciate your feedback regarding the quality of the product. We take all customer feedback seriously and will use it to improve our products in the future.

If you have any further concerns or questions, please do not hesitate to reach out to our customer service team. They will be happy to assist you in any way they can.

Thank you again for your review and for choosing our brand. We hope to have the opportunity to serve you better in the future.

Best regards,

AI customer agent


In [7]:
prompt = f"""
你是一位客户服务的AI助手。
你的任务是给一位重要客户发送邮件回复。
根据客户通过“```”分隔的评价，生成回复以感谢客户的评价。提醒模型使用评价中的具体细节
用简明而专业的语气写信。
作为“AI客户代理”签署电子邮件。
客户评论：
```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt)
print(response)

尊敬的客户，

感谢您对我们产品的评价。我们非常重视您的反馈，对于您提到的问题，我们深表歉意。我们会尽快采取措施，改进我们的产品质量和服务。

我们很高兴您对我们的交付速度感到满意。同时，我们也非常感谢您分享了您对我们产品的使用方法和技巧。我们会将这些细节纳入我们的产品说明中，以帮助更多的客户更好地使用我们的产品。

如果您有任何其他问题或建议，请随时联系我们的客服团队。我们将竭诚为您服务。

此致，

AI客户代理


## 三、使用温度系数

接下来，我们将使用语言模型的一个称为“温度”的参数，它将允许我们改变模型响应的多样性。您可以将温度视为模型探索或随机性的程度。

例如，在一个特定的短语中，“我的最爱食品”最有可能的下一个词是“比萨”，其次最有可能的是“寿司”和“塔可”。因此，在温度为零时，模型将总是选择最有可能的下一个词，而在较高的温度下，它还将选择其中一个不太可能的词，在更高的温度下，它甚至可能选择塔可，而这种可能性仅为五分之一。您可以想象，随着模型继续生成更多单词的最终响应，“我的最爱食品是比萨”将会与第一个响应“我的最爱食品是塔可”产生差异。因此，随着模型的继续，这两个响应将变得越来越不同。

一般来说，在构建需要可预测响应的应用程序时，我建议使用温度为零。在所有课程中，我们一直设置温度为零，如果您正在尝试构建一个可靠和可预测的系统，我认为您应该选择这个温度。如果您尝试以更具创意的方式使用模型，可能需要更广泛地输出不同的结果，那么您可能需要使用更高的温度。

In [8]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [9]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We are sorry to hear that you experienced a price increase on the product you purchased in December. We understand your frustration and apologize for any inconvenience this may have caused.

If you have any further issues, please do not hesitate to reach out to our customer service team. They will be more than happy to assist you in any way they can.

We appreciate your loyalty and hope that you will continue to enjoy our products.

Sincerely,

AI customer agent


In [10]:
prompt = f"""
你是一名客户服务的AI助手。
你的任务是给一位重要的客户发送邮件回复。
根据通过“```”分隔的客户电子邮件生成回复，以感谢客户的评价。
如果情感是积极的或中性的，感谢他们的评价。
如果情感是消极的，道歉并建议他们联系客户服务。
请确保使用评论中的具体细节。
以简明和专业的语气写信。
以“AI客户代理”的名义签署电子邮件。
客户评价：```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

尊敬的客户，

非常感谢您对我们产品的评价。我们很抱歉您在经历使用过程中遇到了问题。我们会尽快处理和改进产品的质量问题。我们希望您能重新联系我们的客户服务部门，我们将竭诚为您提供帮助和解决方案。

同时，我们感谢您对我们品牌的支持和忠诚。我们一直在努力提高我们的产品品质和客户服务。如果您有任何其他问题或建议，请随时联系我们。

再次感谢您的评价，期待您的回复。

AI客户代理


在温度为零时，每次执行相同的提示时，您应该期望获得相同的完成。而使用温度为0.7，则每次都会获得不同的输出。

所以，您可以看到它与我们之前收到的电子邮件不同。让我们再次执行它，以显示我们将再次获得不同的电子邮件。

因此，我建议您自己尝试温度，以查看输出如何变化。总之，在更高的温度下，模型的输出更加随机。您几乎可以将其视为在更高的温度下，助手更易分心，但也许更有创造力。

李鲁鲁：试一下台词生成

In [11]:
prompt = f"""
改写以下每一句文本，补充一些台词，使其成为符合"灼眼的夏娜"的语气的台词: 
```1. 我好饿。```
```2. 今天中午吃什么？```
```3. 冰箱里还有吃的东西吗？```
```4. 我现在有点饿，100块钱的外卖应该不够吃。```
"""
response = get_completion(prompt)
print(response)

1. "我好饿啊，这肚子咕咕叫的声音都快把我吓到了！"
2. "今天中午吃什么呢？难道又是那家无聊的快餐店？"
3. "冰箱里还有吃的东西吗？不要告诉我只有那些过期的泡面和薯片！"
4. "我现在有点饿，100块钱的外卖应该不够吃吧。这种时候，还是得靠自己的厨艺啊！"


In [12]:
prompt = f"""
改写以下每一句文本，补充一些台词，使其成为符合"灼眼的夏娜"的语气的台词: 
```1. 我好饿。```
```2. 今天中午吃什么？```
```3. 冰箱里还有吃的东西吗？```
```4. 我现在有点饿，100块钱的外卖应该不够吃。```
"""
response = get_completion(prompt, temperature=0.75)
print(response)

1. 我的能量消耗殆尽，需要休养生息。
2. 今天午餐应该选一份能量充沛的菜肴。
3. 家里的储备食品是否还能够保证我的能量需求？
4. 空虚的胃部发出了饥饿的信号，但我可不会轻易地放弃我的力量。这100块钱的外卖也许只能填满我的胃，却无法满足我的力量需求。


In [13]:
prompt = f"""
改写以下每一句文本，补充一些台词，使其成为符合"灼眼的夏娜"的语气的台词: 
```1. 我好饿。```
```2. 今天中午吃什么？```
```3. 冰箱里还有吃的东西吗？```
```4. 我现在有点饿，100块钱的外卖应该不够吃。```
"""
response = get_completion(prompt, temperature=0.75)
print(response)

1. 我的食欲正在燃烧，欲求不满啊！
2. 夏娜酱的午餐应该是什么好呢？
3. 那厨房还有没有什么能够填饱肚子的东西？
4. 唉，看来今天只能继续靠我的火力了。100块钱的外卖？还不如让我把它们全部烧掉！


李鲁鲁：感觉调温度似乎感觉用处不大，如果是我我还是选择更好的提示prompt的随机组合去做随机性，而不是去用温度